# Phi-4 Мини ONNX Туторијал за Паралелно Позивање Функција

Овај нотебук демонстрира како да користите Phi-4 Мини са ONNX Runtime GenAI за паралелно позивање функција. Позивање функција омогућава моделу да интелигентно користи спољашње алате и API-је на основу корисничких захтева.

## Преглед

У овом туторијалу ћете научити како да:
- Подесите Phi-4 Мини са ONNX Runtime GenAI
- Дефинишете шеме функција за резервацију летова и хотела
- Користите вођену генерацију са Lark граматиком за структурисани излаз
- Извршите паралелне позиве функција за сложене сценарије резервације путовања

## Предуслови

Пре него што покренете овај нотебук, уверите се да сте:
- Преузели Phi-4 Мини ONNX модел
- Инсталирали пакет `onnxruntime-genai`
- Основно разумели концепте позивања функција


## Корак 1: Увоз потребних библиотека

Прво ћемо увозити неопходне библиотеке за имплементацију позивања функција.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Корак 2: Подешавање и конфигурација модела

Сада ћемо конфигурисати Phi-4 Mini ONNX модел. Обавезно замените путању са стварним директоријумом вашег модела.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Корак 3: Конфигуришите параметре генерисања

Подесите параметре генерисања како бисте контролисали понашање излаза модела. Ова подешавања обезбеђују детерминистичке и усмерене одговоре за позивање функција.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Корак 4: Дефинишите доступне функције

Овде дефинишемо функције које наш AI асистент може да позове. У овом примеру, имамо две функције повезане са путовањем:
1. **booking_flight_tickets**: За резервацију летова између аеродрома
2. **booking_hotels**: За резервацију смештаја у хотелима

Дефиниције функција прате формат шеме за позивање функција OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Корак 5: Помоћне функције за генерисање граматике

Ове помоћне функције претварају дефиниције наших функција у формат граматике Lark, који користи ONNX Runtime GenAI за вођено генерисање. Ово осигурава да модел производи исправне позиве функција у одговарајућем JSON формату.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Корак 6: Тестирање генерисања граматике

Хајде да тестирамо наше функције за генерисање граматике како бисмо видели како претварају дефиниције алата у одговарајући формат.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Корак 7: Припрема системског упутства и генератора

Сада ћемо направити системско упутство које обавештава модел о доступним алатима и подесити генератор са параметрима за вођену генерацију.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Корак 8: Иницијализујте генератор са вођеним генерисањем

Сада ћемо креирати генератор са нашим конфигурисаним параметрима и применити граматику Lark за вођено генерисање.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Корак 9: Тестирање паралелног позивања функција

Сада ћемо тестирати нашу поставку са сложеним сценаријем резервације путовања који захтева позивање више функција.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Генерисање позива функција

Модел ће сада генерисати структурисане позиве функција на основу захтева корисника. Захваљујући вођеном генерисању, излаз ће бити у важећем JSON формату који се може директно извршити.

**Очекивани излаз**: Модел треба да генерише позиве функција за:
1. `booking_flight_tickets` са детаљима за лет од Пекинга (PEK) до Париза (CDG)
2. `booking_hotels` са детаљима за смештај у Паризу

Покрените ћелију испод да бисте видели уживо генерисање:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Закључак

🎉 **Честитамо!** Успешно сте имплементирали паралелно позивање функција са Phi-4 Mini користећи ONNX Runtime GenAI.

### Шта сте научили:

1. **Подешавање модела**: Како конфигурисати Phi-4 Mini са ONNX Runtime GenAI
2. **Дефинисање функција**: Како дефинисати шеме функција за позивање AI функција
3. **Вођена генерација**: Како користити Lark граматику за структурисану генерацију излазних података
4. **Паралелно позивање функција**: Како обрадити сложене захтеве који захтевају више позива функција

### Кључне предности:

- ✅ **Структурисан излаз**: Вођена генерација обезбеђује валидне JSON позиве функција
- ✅ **Паралелна обрада**: Обрада више позива функција у једном захтеву
- ✅ **Високе перформансе**: ONNX Runtime пружа оптимизовану инференцију
- ✅ **Флексибилна шема**: Лако додавање или модификовање дефиниција функција

### Ресурси:

- [Phi-4 Mini документација](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI документација](https://onnxruntime.ai/docs/genai/)
- [Најбоље праксе за позивање функција](https://platform.openai.com/docs/guides/function-calling)



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако тежимо тачности, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква погрешна тумачења или неспоразуме који могу произаћи из коришћења овог превода.
